In [1]:
def minimum(*values, clip=None):
    m = min(values)
    if clip is not None:
         m = clip if clip > m else m
    return m

minimum(1, 5, 2, -5, 100)

-5

In [10]:
def recv(maxsize, *, block):
    'Receives a message'
    pass

recv(10, block=10)

In [11]:
help(recv)

Help on function recv in module __main__:

recv(maxsize, *, block)
    Receives a message



In [15]:
def add(x:int, y:int) -> int:
    return x + y

add('x','y')

'xy'

In [22]:
def spam(a, b=_no_value):
     if b is _no_value:
        print('No b value supplied')
  
spam(1)

NameError: name '_no_value' is not defined

In [23]:
from multiprocessing import Pool

_func = None

def worker_init(func):
    global _func
    _func = func
  
def worker(x):
    return _func(x)

def xmap(func, iterable, processes=None):
    with Pool(processes, initializer=worker_init, initargs=(func,)) as p:
        return p.map(worker, iterable)
  
print(xmap(lambda x: x ** 2, range(100)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


In [26]:
add = lambda x ,y: x + y
add(2, 3)

5

In [31]:
names = ['David Beazley', 'Brian Jones']
sorted(names, key=lambda name: name.split()[-1].lower())

['David Beazley', 'Brian Jones']

In [36]:
funcs = [lambda x, n=n: x+n for n in range(5)]
len(funcs)
for f in funcs: print(f(0))

0
1
2
3
4


In [ ]:
from functools import partial

def spam(a, b, c, d):
    print(a, b, c, d)
    
s = partial(spam, 1, 3, 7)    
s(2)

In [15]:
def output_result(result, log=None):
    if log is not None:
        log.debug("Got: %r", result)
        
def add(x, y):
    return x + y

if __name__ == '__main__':
    import logging
    from multiprocessing import Pool
    from functools import partial
    
    logging.basicConfig(level=logging.DEBUG)
    log = logging.getLogger('test')
    
    p = Pool()
    p.apply_async(add, (3,4), callback=partial(output_result, log=log))
    p.close()
    p.join()

DEBUG:test:Got: 7


In [ ]:
from socketserver import StreamRequestHandler, TCPServer

class EchoHandler(StreamRequestHandler):
    def handle(self):
        for line in self.rfile:
            self.wfile.write(b'GOT:' + line)
        
serv = TCPServer(('', 15000), EchoHandler)
serv.se
#serv.serve_forever()

In [ ]:
from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template
        
    def open(self, **kargs):
        return urlopen(self.template.format_map(kargs))
    
yahoo = UrlTemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')

for line in yahoo.open(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

In [ ]:
import sys
sys.path

In [4]:
def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)
    
def print_result(result):
    print('Got:', result) 

def add(x, y):
    return x + y

apply_async(add, (2,3), callback=print_result)

Got: 5


In [6]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0
    def handler(self, result):
        self.sequence += 1
        apply_async(add, (2, 3), callback=r.handler)

r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler)

[1] Got: 5


In [7]:
apply_async(add, (2, 3), callback=r.handler)

[2] Got: 5


In [8]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        apply_async(add, (2, 3), callback=r.handler) 
    return handler

handler = make_handler()
apply_async(add, (2, 3), callback=handler)

[3] Got: 5


In [14]:
class SequenceNo:
    def __init__(self):
        self.sequence = 0

def handler(result, seq):
    seq.sequence += 1
    print('[{}] Got: {}'.format(seq.sequence, result))

seq = SequenceNo()
from functools import partial
apply_async(add, (2, 3), callback=partial(handler, seq=seq))

[1] Got: 5


In [16]:
apply_async(add, (2, 3), callback=lambda r: handler(r, seq))

[3] Got: 5


In [22]:
from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

def inlined_async(func):
    @wraps(func)
    def wrapper(*args): 
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper     

def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

test()    

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


In [25]:
def sample():
    n = 0
    
    # Closure function
    def func():
        print('n=', n)
    
    # Accessor methods for n
    def get_n():     
        return n
    
    def set_n(value):
        nonlocal n
        n = value
    
    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func

f = sample()
f.set_n(10)
f.get_n()

10

In [26]:
import sys

class ClosureInstance:
    
    def __init__(self, locals=None):
        if locals is None:
            locals = sys._getframe(1).f_locals
 
        # Update instance dictionary with callables
        self.__dict__.update((key,value) for key, value in locals.items() if callable(value) )
 
    # Redirect special methods
    def __len__(self):
        return self.__dict__['__len__']()
    
    

In [31]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
 
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)
    
p = Pair(3,42)
p

Pair(3, 42)

In [32]:
print(p)

(3, 42)


In [33]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.sock = None
    
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None
        
conn = LazyConnection(('www.python.org', 80))
with conn as s:
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))

In [35]:
class Person:
    
    def __init__(self, first_name):
        self.first_name = first_name

    # Getter function
    @property
    def first_name(self):
        return self._first_name

    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")
        
a = Person("Guido")
a.first_name

'Guido'